In [30]:
# ติดตั้งไลบรารีที่จำเป็น
!pip install requests ujson xmltodict

import requests
import ujson as json
import xmltodict

In [31]:
# URL และค่าคงที่ที่เกี่ยวข้อง
ERROR_KEY = u'error'
_WIKIFIER_WIKIFY_URL = u"http://www.wikifier.org/annotate-article" # URL ของ wikifier
_WIKIFIER_SR_URL = "http://www.wikifier.org/get-cosine-similarity" # URL สำหรับการคำนวณ Semantic Relatedness (ความสัมพันธ์เชิงความหมาย) ระหว่างหัวข้อสองหัวข้อ (concept pair) ที่เราส่งเข้าไปให้ API ใช้งานในฟังก์ชัน


In [32]:
def get_wikifier_wikify_response(text, api_key="pwdxjzlnmegtncwumufswitkbefgxb", df_ignore=0, words_ignore=0):
    """
    ดึงข้อมูลคำศัพท์และความหมายจาก Wikifier API
    """
    params = {
        "text": text,
        "userKey": api_key,
        "nTopDfValuesToIgnore": df_ignore,            # เมินคำที่มีความถี่สูงสุด เพื่อกันปัญหาคำเช่น The, is, are and etc.
        "nWordsToIgnoreFromList": words_ignore   # เมินคำที่มักจะไม่สำคัญ อิงตาม list คำของ wikipedia  (ไม่ทราบว่ามีคำไหนบ้าง )
    }
    r = requests.post(_WIKIFIER_WIKIFY_URL, params)
    if r.status_code == 200: # ทำการตรวจสอบ response ว่าเท่ากับ 200 ไหม ถ้าใช่แปลว่าร้องขอสำเร็จ
    # รหัสสถานะ HTTP 100 ดำเนินการต่อ 200 สำเร็จ 300 เปลี่ยนเส้นทาง 400 ฝั่งไคลแอนด์ผิดพลาด 500 ฝั่ง เซิฟเวอร์ผิดพลาด
        resp = json.loads(r.content) # แปลงข้อมูล response จาก JSON เป็น Python object:
        if ERROR_KEY in resp:
            raise ValueError(f"Error in response: {resp[ERROR_KEY]}")
        return resp
    else:
        raise ValueError(f"HTTP status code 200 expected, got status code {r.status_code} instead")

{ "annotation" : url -> ลิงก์ไปบทความ

lang -> ภาษา

pageRank -> บอกถึงความสำคัญของคำในบริบทของข้อความ

cosine -> บอกถึงระดับความสัมพันธ์ระหว่างคำและหัวข้อที่ระบุ

secLang -> บอกภาษาของบทความ

secTitle -> ชื่อบทความ

secUrl -> ลิงก์เสริม

wikiDataItemId -> รหัสของบทความ

wikiDataClasses -> category ของบทความประกอบด้วย {รหัส} และ {ชื่อภาษาอังกฤษ}

In [33]:
get_wikifier_wikify_response("AI" )

{'annotations': [{'title': 'Artificial intelligence',
   'url': 'http://en.wikipedia.org/wiki/Artificial_intelligence',
   'lang': 'en',
   'pageRank': 0.1931977578054374,
   'cosine': 0.4603672456950956,
   'secLang': 'en',
   'secTitle': 'Artificial intelligence',
   'secUrl': 'http://en.wikipedia.org/wiki/Artificial_intelligence',
   'wikiDataItemId': 'Q11660',
   'wikiDataClasses': [{'itemId': 'Q268592', 'enLabel': 'industry'},
    {'itemId': 'Q112057532', 'enLabel': 'type of technology'},
    {'itemId': 'Q123370638', 'enLabel': 'branch of computer science'},
    {'itemId': 'Q125161275', 'enLabel': 'type of intelligence'},
    {'itemId': 'Q16887380', 'enLabel': 'group'},
    {'itemId': 'Q21146257', 'enLabel': 'type'},
    {'itemId': 'Q2465832', 'enLabel': 'branch of science'},
    {'itemId': 'Q1047113', 'enLabel': 'specialty'},
    {'itemId': 'Q99527517', 'enLabel': 'collective entity'},
    {'itemId': 'Q16889133', 'enLabel': 'class'},
    {'itemId': 'Q336', 'enLabel': 'science'},


In [34]:
get_wikifier_wikify_response("AI is transforming industries such as healthcare, finance, and education.")

{'annotations': [{'title': 'Artificial intelligence',
   'url': 'http://en.wikipedia.org/wiki/Artificial_intelligence',
   'lang': 'en',
   'pageRank': 0.05820631549294691,
   'cosine': 0.2918208869862075,
   'secLang': 'en',
   'secTitle': 'Artificial intelligence',
   'secUrl': 'http://en.wikipedia.org/wiki/Artificial_intelligence',
   'wikiDataItemId': 'Q11660',
   'wikiDataClasses': [{'itemId': 'Q268592', 'enLabel': 'industry'},
    {'itemId': 'Q112057532', 'enLabel': 'type of technology'},
    {'itemId': 'Q123370638', 'enLabel': 'branch of computer science'},
    {'itemId': 'Q125161275', 'enLabel': 'type of intelligence'},
    {'itemId': 'Q16887380', 'enLabel': 'group'},
    {'itemId': 'Q21146257', 'enLabel': 'type'},
    {'itemId': 'Q2465832', 'enLabel': 'branch of science'},
    {'itemId': 'Q1047113', 'enLabel': 'specialty'},
    {'itemId': 'Q99527517', 'enLabel': 'collective entity'},
    {'itemId': 'Q16889133', 'enLabel': 'class'},
    {'itemId': 'Q336', 'enLabel': 'science'},

In [35]:
get_wikifier_wikify_response("AI is transforming industries such as healthcare, finance, and education.")

{'annotations': [{'title': 'Artificial intelligence',
   'url': 'http://en.wikipedia.org/wiki/Artificial_intelligence',
   'lang': 'en',
   'pageRank': 0.05820631549294691,
   'cosine': 0.2918208869862075,
   'secLang': 'en',
   'secTitle': 'Artificial intelligence',
   'secUrl': 'http://en.wikipedia.org/wiki/Artificial_intelligence',
   'wikiDataItemId': 'Q11660',
   'wikiDataClasses': [{'itemId': 'Q268592', 'enLabel': 'industry'},
    {'itemId': 'Q112057532', 'enLabel': 'type of technology'},
    {'itemId': 'Q123370638', 'enLabel': 'branch of computer science'},
    {'itemId': 'Q125161275', 'enLabel': 'type of intelligence'},
    {'itemId': 'Q16887380', 'enLabel': 'group'},
    {'itemId': 'Q21146257', 'enLabel': 'type'},
    {'itemId': 'Q2465832', 'enLabel': 'branch of science'},
    {'itemId': 'Q1047113', 'enLabel': 'specialty'},
    {'itemId': 'Q99527517', 'enLabel': 'collective entity'},
    {'itemId': 'Q16889133', 'enLabel': 'class'},
    {'itemId': 'Q336', 'enLabel': 'science'},

In [36]:
def get_top_5_wikifier_response(text, api_key="pwdxjzlnmegtncwumufswitkbefgxb", df_ignore=0, words_ignore=0):
    """
    Gets the top 5 results from the Wikifier API response.
    """
    response = get_wikifier_wikify_response(text, api_key, df_ignore, words_ignore)

    if "annotations" in response:
        annotations = response["annotations"]

        annotations.sort(key=lambda x: float(x.get("pageRank", 0)), reverse=True)
        return annotations[:5]  # Return the top 5 annotations
    else:
        return []


In [37]:
top_5_results = get_top_5_wikifier_response("AI is transforming industries such as healthcare, finance, and education.")
top_5_results

[{'title': 'Artificial intelligence',
  'url': 'http://en.wikipedia.org/wiki/Artificial_intelligence',
  'lang': 'en',
  'pageRank': 0.05820631549294691,
  'cosine': 0.2918208869862075,
  'secLang': 'en',
  'secTitle': 'Artificial intelligence',
  'secUrl': 'http://en.wikipedia.org/wiki/Artificial_intelligence',
  'wikiDataItemId': 'Q11660',
  'wikiDataClasses': [{'itemId': 'Q268592', 'enLabel': 'industry'},
   {'itemId': 'Q112057532', 'enLabel': 'type of technology'},
   {'itemId': 'Q123370638', 'enLabel': 'branch of computer science'},
   {'itemId': 'Q125161275', 'enLabel': 'type of intelligence'},
   {'itemId': 'Q16887380', 'enLabel': 'group'},
   {'itemId': 'Q21146257', 'enLabel': 'type'},
   {'itemId': 'Q2465832', 'enLabel': 'branch of science'},
   {'itemId': 'Q1047113', 'enLabel': 'specialty'},
   {'itemId': 'Q99527517', 'enLabel': 'collective entity'},
   {'itemId': 'Q16889133', 'enLabel': 'class'},
   {'itemId': 'Q336', 'enLabel': 'science'},
   {'itemId': 'Q105948247', 'enLab

In [38]:
titles = [result.get('secTitle', '') for result in top_5_results]
titles


['Artificial intelligence',
 'Health care',
 'Finance',
 'Education',
 'Information technology']